# Threads

When you run a Python program the flow of execution of the code starts at the top of files and works it's way downwards a line at a time.

The flow of execution will change if you call a function as the code will go  here to keep executing.

The model of execution is quite simple but has a major limitation, what if you want to do more than one thing at once?

For many modern operating systems to be useful they have to be able to run multiple tasks at the same time. To do this threading is provided as a way to be able to run multiple programs at once.

One of the simplest concurrency paradigms is the notion of the thread, this is supported in the Python standard library in the `threading` module https://docs.python.org/3/library/threading.html

In [3]:
# Python's thread library
import threading

## What is a thread

A thread is effectively a separate flow of control of execution of code. Threads do not provide a separate process for execution so there's shared state with multiple flows of execution. (Note that thread local data is provided in case a thread needs it's own memory)

To do this there's 2 things that the operating system will have to provide:

1. task scheduler
2. task runner

Given that more than one operation is possible at once the operating system has to have a way of dealing with not just *how* to execute code but *when* to execute that code as well. This detail ends up being an important aspect of heavily concurrent code because the cost of switching between multiple threads is non-zero.

## Creating a thread

We can create a thread using the `threading.Thread` by passing in a function to run on the thread as a target. We then have to run the thread with `.start()`.

For example:

In [ ]:
from time import sleep
from threading import Thread
t = Thread(target=lambda: sleep(30))
t.start() # Start executing the target function on the thread
t.join()  # Wait for Thread to finish

### Exercise

- Create 1000 threads that sleep
- Look at the memory usage for your code in your operating systems (hint: `os.getpid()` might help you find the current process more easily)

Note that you may want to not run this in Jupyter-notebook.

In [ ]:
# Solution: scripts/thread_memory_usage.py
import os
from time import sleep
from threading import Thread
print(f'PID = {os.getpid()}')

NUM_THREADS = 1000
threads = [Thread(target=lambda: sleep(60)) for _ in range(NUM_THREADS)]
[t.start() for t in threads]
[t.join() for t in threads]

## Finding the memory usage

top/htop are a good way to show memory usage from the command line on `*nix` systems such as MacOS and Linux.

`top -p PID` will give you info for the process with process number PID.

Advanced: the files `/proc/PID/maps` and `/proc/PID/smaps` contain more detailed memory usage.

So when I ran my script I had the PID 7170:

In [ ]:
%%bash
$ top -p 7170
  PID USER
PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND
7170 janis     20   0 13.795g  22512   5572 S   0.0  0.3   0:00.24 python3

As you can see there's a huge amount of virtual memory assigned here due to each thread having memory assigned to it. Very little of this memory is actually in active use however.

### 32 bit architectures note:

Note that creating a large number of threads is a substantial issue on 32 bit architectures, you simply run out of memory address space quickly if you have a one-thread-per-task architecture and have a lot of tasks to run. I have seen this be an issue on high throughput web servers in the past that were running on 32bit operating systems. If you can use an alternative approach such as `asyncio` you will not exhaust the address space.

## Simple thread example: slow IO

IO operations, especially those involving a network connection are very slow, let's put those on their own thread so they don't hold up everything else.

In [7]:
# Slow
import time

messaged_processed = 0

def delayed_print(message: str, delay: int):
    """Print the given message after delaying first"""
    time.sleep(delay)
    print(message)
    global messaged_processed
    messaged_processed += 1

delayed_print("hello", 1)
delayed_print("world", 2)
print(f"We processed {messaged_processed} messages!")

hello
world
We processed 2 messages!


In the slow example above we got what we expected, "hello" printed out followed by "world" a second later. However this completely hold up the process, nothing can happen while we are waiting. We also have a global variable that keeps track of how many messages the system processed, this will turn out to cause problems as we will see shortly.

## Running tasks on a thread

We can run a function on a thread using the
`threading.Thread` interface:

In [10]:
import time
from threading import Thread

messaged_processed = 0

def delayed_print(message: str, delay: int):
    """Print the given message after delaying first"""
    time.sleep(delay)
    print(message)
    global messaged_processed
    messaged_processed += 1

threads = [
    Thread(target=delayed_print, args=("hello", 1)),
    Thread(target=delayed_print, args=("world", 2)),
]

print(f"{len(threads)} messages to process...")
# Start our threads
[t.start() for t in threads]
# Wait for the threads to finish
[t.join() for t in threads]
print(f"Processed {messaged_processed} messages")

2 messages to process...
hello
world
Processed 2 messages


## Which thread are we running on?

sometimes you want to know what thread you are running on:

In [8]:
threading.get_ident()

140159479781184

In [2]:
threading.current_thread().name

'MainThread'

### Exercise

- Create a function that prints out a message `msg` after a specified waiting time parameter `delay`. This can take keyword arguments that modify what's printed, eg `uppercase=True` makes the message uppercase before printing it
- Run this function from a thread

In [15]:
def delayed_print(message: str, delay: int, *, uppercase=False):
    """Print the given message after delaying first"""
    time.sleep(delay)
    if uppercase:
        message = message.upper()
    print(message)
    global messaged_processed
    messaged_processed += 1

t = Thread(target=delayed_print, args=("hello world", 1), kwargs={"uppercase":True})
t.start()
t.join()

HELLO WORLD


## Locks

When you have a single thread of execution of code you don't have to worry about 2 separate cores of execution accessing the same piece of data at once, the bugs that come up because due to ordering of execution are known as race conditions and are some of the most nasty bugs that software engineers will face.

Acquiring a lock is a way in which you can make sure that only one thread is going to access a variable at the same time. Note that a lock *prevents* other threads of execution from also accessing the same variable.

*If you can provably make your program work in parallel without locks it is highly advantageous to not use locks*. Making programs that will be *provably* correct in a concurrent setting without using locks is hard, which is why for example the CPython interpreter still extensively uses a global interpreter lock to make sure internal consistency is maintained. You may wish to look at Non-blocking algorithms and lock free programming techniques, though this is beyond the scope of this current document.

In [1]:
import time
from threading import Thread

messaged_processed = 0

def delayed_message_processing(message: str, delay: int):
    time.sleep(delay)
    messaged_processed = messaged_processed + 1

threads = []
for _ in range(1000):
    threads.append(Thread(target=delayed_message_processing, args=("test", 1)))

print(f"{len(threads)} messages to process...")
# Start our threads
[t.start() for t in threads]
# Wait for the threads to finish
[t.join() for t in threads]
print(f"Processed {messaged_processed} messages")

1000 messages to process...


### Deadlocking

If A is locked and is waiting on B and B is locked waiting on A we have a condition known as a deadlock. Because both threads are simultaneously waiting on each other no progress can be made. Try to avoid such situations in your code. Here's an example from the `concurrent.futures` documentation:

In [17]:
from concurrent.futures import ThreadPoolExecutor
import time
def wait_on_b():
    time.sleep(5)
    print(b.result())  # b will never complete because it is waiting on a.
    return 5

def wait_on_a():
    time.sleep(5)
    print(a.result())  # a will never complete because it is waiting on b.
    return 6


executor = ThreadPoolExecutor(max_workers=2)
a = executor.submit(wait_on_b)
b = executor.submit(wait_on_a)
print(a.running())
print(b.running())
time.sleep(10)
print(a.running())
print(b.running())

True
True
True
True


## Thread pools

Creating and destroying threads can be much more expensive than whatever it is you are actually trying to do. Since there's a maximum number of threads your processor can actually process at any given time it makes sense to not create a lot more threads than whatever number that is. Creating a pool of threads can substantially improve performance (but not always see GIL example below for a case where more threads means worse performance)

## The global interpreter lock (GIL)

If more than one thread of execution could access internal Python data structures some very nasty bugs could occur in the interpreter itself which could lead to interpreter crashing, or worse, incorrect computations from occurring. So for the interpreter to protect it's internal state it acquires a global lock, this is very often referred to in the community by the acronym GIL.

Quick summary:

- The GIL prevents the CPython interpreter from executing more than one piece of bytecode at the same time
- Actions like `time.sleep()` don't run bytecode and hence do not hold this lock
- IO also does not hold the interpreter lock
- You will notice that in your operating systems task manager CPython programs tend to use only one core unless explicitly designed otherwise (see our other materials for how you can do this in your
code)

Threading will allow you to get concurrency in cases where bytecode is
*not* executing. So for most operations that wait on IO you can get better
processor utilization and less latency by letting the waiting occur in threads.
(Note that for IO tasks the Asyncio library and approach may be substantially
easier to manage)

There's many articles about the GIL, but the one of the main
issues/gotcha's is that if you need parallel computation with multiple bytecode
operations being executed simultaneously if the GIL is acquired it will prevent
you from being able to achieve this.

*Running parallel computations in threads
with the GIL being acquired will decrease performance compared to not using
threads at all*. This is because you will incur thread scheduling overhead
without any additional instructions being able to be executed, the more threads
you add when the GIL is the bottleneck the worse your performance will get.

In [29]:
import math
import os
def slow_example(run_number):
    """Run an expensive function that doesn't release the GIL"""
    print(run_number)
    print("Executing our Task on Process: {}".format(os.getpid()))
    y = [1]*10000000
    [math.exp(i) for i in y]

In [30]:
%%timeit
NUM_TRIALS = 4

print("Start")
list(map(slow_example, list(range(NUM_TRIALS))))
print("End")

Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task o

In [36]:
%%timeit
import math
from concurrent.futures import ThreadPoolExecutor
NUM_TRIALS = 4

print("Start")
with ThreadPoolExecutor(max_workers=4) as executor:
    result = executor.map(slow_example, range(NUM_TRIALS))
print("End")

Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task on Process: 9938
1
Executing our Task on Process: 9938
2
Executing our Task on Process: 9938
3
Executing our Task on Process: 9938
End
Start
0
Executing our Task o

## What to do instead?

Notice how adding the thread pool there made things slower not faster! With more threads running this just gets worse (try it out). Since the `ProcessPoolExecutor` in `concurrent.futures` shares the same API as the `ThreadPoolExecutor` we can use this as a drop in replacement for the sake of this example.

In [35]:
%%timeit
import math
from concurrent.futures import ProcessPoolExecutor
NUM_TRIALS = 4

print("Start")
with ProcessPoolExecutor(max_workers=4) as executor:
    result = executor.map(slow_example, range(NUM_TRIALS))
print("End")

Start
0
1
2
3
Executing our Task on Process: 12023
Executing our Task on Process: 12025
Executing our Task on Process: 12022
Executing our Task on Process: 12024
End
Start
2
1
3
0
Executing our Task on Process: 12047
Executing our Task on Process: 12046
Executing our Task on Process: 12048
Executing our Task on Process: 12045
End
Start
1
0
3
2
Executing our Task on Process: 12068
Executing our Task on Process: 12067
Executing our Task on Process: 12069
Executing our Task on Process: 12070
End
Start
0
1
3
2
Executing our Task on Process: 12090
Executing our Task on Process: 12091
Executing our Task on Process: 12093
Executing our Task on Process: 12092
End
Start
1
3
0
2
Executing our Task on Process: 12113
Executing our Task on Process: 12114
Executing our Task on Process: 12112
Executing our Task on Process: 12115
End
Start
1
3
0
2
Executing our Task on Process: 12134
Executing our Task on Process: 12135
Executing our Task on Process: 12137
Executing our Task on Process: 12136
End
Star